Assignnement 2: Prediction and Classification

Due: Thursday, April 30, 2015 11:59 PM
# lAMA

# Introduction
Problem 1 is optional - for extra credit!
Problems 2 and 3 will be graded for the Lab 2.  


In this assignment you will be using regression and classification to explore different data sets.  

**First**: You will use data from before 2002 in the [Sean Lahman's Baseball Database](http://seanlahman.com/baseball-archive/statistics) to create a metric for picking baseball players using linear regression.  This database contains the "complete batting and pitching statistics from 1871 to 2013, plus fielding statistics, standings, team stats, managerial records, post-season data, and more". [Documentation provided here](http://seanlahman.com/files/database/readme2012.txt).

!["Sabermetrics Science"](http://saberseminar.com/wp-content/uploads/2012/01/saber-web.jpg)
http://saberseminar.com/wp-content/uploads/2012/01/saber-web.jpg

**Second**: You will use the famous [iris](http://en.wikipedia.org/wiki/Iris_flower_data_set) data set to perform a $k$-neareast neighbor classification using cross validation.  While it was introduced in 1936, it is still [one of the most popular](http://archive.ics.uci.edu/ml/) example data sets in the machine learning community. Wikipedia describes the data set as follows: "The data set consists of 50 samples from each of three species of Iris (Iris setosa, Iris virginica and Iris versicolor). Four features were measured from each sample: the length and the width of the sepals and petals, in centimetres." Here is an illustration what the four features measure:

!["iris data features"](http://sebastianraschka.com/Images/2014_python_lda/iris_petal_sepal.png)
http://sebastianraschka.com/Images/2014_python_lda/iris_petal_sepal.png

**Third**: You will investigate the influence of higher dimensional spaces on the classification using another standard data set in machine learning called the The [digits data set](http://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_digits.html).   Each digit is represented by an 8x8 pixel image patch. In addition, the gray values are restricted to 16 different values (4 bit). 



## Load Python modules

In [385]:
# prepare the notebook for matplotlib
%matplotlib inline 

import requests 
import StringIO
import zipfile
import numpy as np
import pandas as pd # pandas
import matplotlib.pyplot as plt # module for plotting 

# If this module is not already installed, you may need to install it. 
# You can do this by typing 'pip install seaborn' in the command line
import seaborn as sns 

import sklearn
import sklearn.datasets
import sklearn.cross_validation
import sklearn.decomposition
import sklearn.grid_search
import sklearn.neighbors
import sklearn.metrics

# Problem 1: Sabermetrics

Using data preceding the 2002 season pick 10 offensive players keeping the payroll under $20 million (assign each player the median salary). Predict how many games this team would win in a 162 game season.  

In this problem we will be returning to the [Sean Lahman's Baseball Database](http://seanlahman.com/baseball-archive/statistics).  From this database, we will be extract five data sets containing information such as yearly stats and standing, batting statistics, fielding statistics, player names, player salaries and biographical information. You will explore the data in this database from before 2002 and create a metric for picking players. 

#### Problem 1(a) 

Load in [these CSV files](http://seanlahman.com/files/database/lahman-csv_2014-02-14.zip) from the [Sean Lahman's Baseball Database](http://seanlahman.com/baseball-archive/statistics). For this assignment, we will use the 'Teams.csv', 'Batting.csv', 'Salaries.csv', 'Fielding.csv', 'Master.csv' tables. Read these tables into separate pandas DataFrames with the following names. 

CSV file name | Name of pandas DataFrame
:---: | :---: 
Teams.csv | teams
Batting.csv | players
Salaries.csv | salaries
Fielding.csv | fielding
Master.csv | master

In [386]:
### Your code here ###
teams = pd.read_csv('data/baseball_data/teams.csv')
players = pd.read_csv('data/baseball_data/Batting.csv')
salaries = pd.read_csv('data/baseball_data/Salaries.csv')
fielding = pd.read_csv('data/baseball_data/Fielding.csv')
master = pd.read_csv('data/baseball_data/Master.csv', index_col=0)
master.index.name = None

#### Problem 1(b)

Calculate the median salary for each player and create a pandas DataFrame called `medianSalaries` with four columns: (1) the player ID, (2) the first name of the player, (3) the last name of the player and (4) the median salary of the player. Show the head of the `medianSalaries` DataFrame.   

In [387]:
first_last = master[['nameFirst', 'nameLast']]
#yearID_salary = salaries[['yearID', 'salary', 'playerID']]
yearID_salary = salaries[['salary', 'playerID']]

In [388]:
yearID_salary.index = yearID_salary['playerID'].values
#yearID_salary.drop('playerID', axis=1, inplace=True)
yearID_salary.head()

,salary,playerID
murraed02,1472819,murraed02
lynnfr01,1090000,lynnfr01
ripkeca01,800000,ripkeca01
lacyle01,725000,lacyle01
flanami01,641667,flanami01


In [389]:
salary_first_last = pd.merge(left = yearID_salary, right = first_last,left_index=True, right_index=True, how = "inner")

In [419]:
medianSalaries = salary_first_last.groupby(['nameFirst', 'nameLast', 'playerID'])['salary'].median().reset_index()
medianSalaries.head()

,nameFirst,nameLast,playerID,salary
0,A. J.,Burnett,burneaj01,3650000
1,A. J.,Ellis,ellisaj01,490000
2,A. J.,Griffin,griffaj01,492500
3,A. J.,Hinch,hinchaj01,245000
4,A. J.,Pierzynski,pierzaj01,4000000


#### Problem 1(c)

Now, consider only team/season combinations in which the teams played 162 Games. Exclude all data from before 1947. Compute the per plate appearance rates for singles, doubles, triples, HR, and BB. Create a new pandas DataFrame called `stats` that has the teamID, yearID, wins and these rates.

**Hint**: Singles are hits that are not doubles, triples, nor HR. Plate appearances are base on balls plus at bats.

1(c) Filter the teams data frame so that you only have records for when year > 1947 and number of games > 162.  Plate appearances = BB + AB = Walks (base on balls) + At Bats.  Singles = Hits – (Doubles + Triple+ HomeRun).  Compute “per plate appearance rates for singles” = Singles/Plate Appearances.

PA = AB + BB

In [434]:
year_condition = teams['yearID'] >= 1947
games_condition = teams['G'] >= 162

In [437]:
filtered_teams = teams[year_condition & games_condition]
filtered_teams.head()

,yearID,lgID,teamID,franchID,divID,Rank,G,Ghome,W,L,...,DP,FP,name,park,attendance,BPF,PPF,teamIDBR,teamIDlahman45,teamIDretro
1359,1961,AL,NYA,NYY,NaN,1,163,81.0,109,53,...,180.0,0.98,New York Yankees,Yankee Stadium I,1747725.0,95,93,NYY,NYA,NYA
1360,1961,AL,DET,DET,NaN,2,163,82.0,101,61,...,147.0,0.97,Detroit Tigers,Tiger Stadium,1600710.0,103,102,DET,DET,DET
1361,1961,AL,BAL,BAL,NaN,3,163,82.0,95,67,...,173.0,0.98,Baltimore Orioles,Memorial Stadium,951089.0,96,96,BAL,BAL,BAL
1362,1961,AL,CHA,CHW,NaN,4,163,81.0,86,76,...,138.0,0.98,Chicago White Sox,Comiskey Park,1146019.0,99,97,CHW,CHA,CHA
1364,1961,AL,BOS,BOS,NaN,6,163,82.0,76,86,...,140.0,0.97,Boston Red Sox,Fenway Park II,850589.0,102,103,BOS,BOS,BOS


In [442]:
filtered_teams['PA'] = filtered_teams['BB'] + filtered_teams['AB']

/Users/mgalarny/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [444]:
filtered_teams.columns

Index([u'yearID', u'lgID', u'teamID', u'franchID', u'divID', u'Rank', u'G',
       u'Ghome', u'W', u'L', u'DivWin', u'WCWin', u'LgWin', u'WSWin', u'R',
       u'AB', u'H', u'2B', u'3B', u'HR', u'BB', u'SO', u'SB', u'CS', u'HBP',
       u'SF', u'RA', u'ER', u'ERA', u'CG', u'SHO', u'SV', u'IPouts', u'HA',
       u'HRA', u'BBA', u'SOA', u'E', u'DP', u'FP', u'name', u'park',
       u'attendance', u'BPF', u'PPF', u'teamIDBR', u'teamIDlahman45',
       u'teamIDretro', u'PA'],
      dtype='object')

In [445]:
filtered_teams['Singles'] = filtered_teams['H'] - filtered_teams['2B'] - filtered_teams['3B'] - filtered_teams['HR']

/Users/mgalarny/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [455]:
stats = filtered_teams[['teamID','yearID', 'W', 'Singles', '2B', '3B', 'HR', 'BB']]
stats['Singles'] = stats['Singles']/filtered_teams['PA']
stats['2B'] = stats['2B']/filtered_teams['PA']
stats['3B'] = stats['3B']/filtered_teams['PA']
stats['HR'] = stats['HR']/filtered_teams['PA']
stats['BB'] = stats['BB']/filtered_teams['PA']
stats = stats.reset_index()
stats

/Users/mgalarny/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/mgalarny/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/Users/mgalarny/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

,index,teamID,yearID,W,Singles,2B,3B,HR,BB
0,1359,NYA,1961,109,0.161750,0.031793,0.006555,0.039331,0.088987
1,1360,DET,1961,101,0.165704,0.034488,0.008502,0.028874,0.107956
2,1361,BAL,1961,95,0.161828,0.037446,0.005939,0.024579,0.095843
3,1362,CHA,1961,86,0.176056,0.035375,0.007534,0.022601,0.090075
4,1364,BOS,1961,76,0.162632,0.040780,0.006011,0.018197,0.105118
5,1366,LAA,1961,70,0.147748,0.035708,0.003604,0.030958,0.111548
6,1367,KC1,1961,61,0.164751,0.035982,0.007829,0.014993,0.096618
7,1377,NYA,1962,96,0.167148,0.038536,0.004656,0.031952,0.093770
8,1378,MIN,1962,91,0.161997,0.034622,0.006280,0.029791,0.104509
9,1379,LAA,1962,86,0.159482,0.038027,0.005737,0.022455,0.098672


#### Problem 1(d)

Is there a noticeable time trend in the rates computed computed in Problem 1(c)? 

In [393]:
### Your code here ###

#### Problem 1(e) 

Using the `stats` DataFrame from Problem 1(c), adjust the singles per PA rates so that the average across teams for each year is 0. Do the same for the doubles, triples, HR, and BB rates. 

In [394]:
### Your code here ###

#### Problem 1(f)

Build a simple linear regression model to predict the number of wins from the average adjusted singles, double, triples, HR, and BB rates. To decide which of these terms to include fit the model to data from 2002 and compute the average squared residuals from predictions to years past 2002. Use the fitted model to define a new sabermetric summary: offensive predicted wins (OPW). Hint: the new summary should be a linear combination of one to five of the five rates.


In [395]:
### Your code here ###

** Your answer here: **

#### Problem 1(g)

Now we will create a similar database for individual players. Consider only player/year combinations in which the player had at least 500 plate appearances. Consider only the years we considered for the calculations above (after 1947 and seasons with 162 games). For each player/year compute singles, doubles, triples, HR, BB per plate appearance rates. Create a new pandas DataFrame called `playerstats` that has the playerID, yearID and the rates of these stats.  Remove the average for each year as for these rates as done in Problem 1(e). 

In [396]:
### Your code here ###

Show the head of the `playerstats` DataFrame. 

In [397]:
### Your code here ###

#### Problem 1(h)

Using the `playerstats` DataFrame created in Problem 1(g), create a new DataFrame called `playerLS` containing the player's lifetime stats. This DataFrame should contain the playerID, the year the player's career started, the year the player's career ended and the player's lifetime average for each of the quantities (singles, doubles, triples, HR, BB). For simplicity we will simply compute the avaerage of the rates by year (a more correct way is to go back to the totals). 

In [398]:
### Your code here ###

Show the head of the `playerLS` DataFrame. 

In [399]:
### Your code here ###

#### Problem 1(i)

Compute the OPW for each player based on the average rates in the `playerLS` DataFrame. You can interpret this summary statistic as the predicted wins for a team with 9 batters exactly like the player in question. Add this column to the playerLS DataFrame. Call this colum OPW.

In [400]:
### Your code here ###

#### Problem 1(j)

Add four columns to the `playerLS` DataFrame that contains the player's position (C, 1B, 2B, 3B, SS, LF, CF, RF, or OF), first name, last name and median salary. 

In [401]:
### Your code here ###

Show the head of the `playerLS` DataFrame. 

In [402]:
### Your code here ###

#### Problem 1(k)

Subset the `playerLS` DataFrame for players active in 2002 and 2003 and played at least three years. Plot and describe the relationship bewteen the median salary (in millions) and the predicted number of wins. 

In [403]:
### Your code here ###

#### Problem 1(l)
Pick one players from one of each of these 10 position C, 1B, 2B, 3B, SS, LF, CF, RF, DH, or OF keeping the total median salary of all 10 players below 20 million. Report their averaged predicted wins and total salary.

In [404]:
### Your code here ###

#### Problem 1(m)
What do these players outperform in? Singles, doubles, triples HR or BB?

In [405]:
### Your code here ###

** Your answer here: **

### 

### Use one of the classification methods to predict wheather a player will be an Allstar?

In [406]:
### Your code here ###

## Discussion for Problem 1

*Write a brief discussion of your conclusions to the questions and tasks above in 100 words or less.*

---

# Problem 2:  $k$-Nearest Neighbors and Cross Validation 

What is the optimal $k$ for predicting species using $k$-nearest neighbor classification 
on the four features provided by the iris dataset.

In this problem you will get to know the famous iris data set, and use cross validation to select the optimal $k$ for a $k$-nearest neighbor classification. This problem set makes heavy use of the [sklearn](http://scikit-learn.org/stable/) library. In addition to Pandas, it is one of the most useful libraries for data scientists. 
For the Iris data set  sklearn provides an extra function to load it - since it is one of the very commonly used data sets.

In [407]:
#load the iris data set


#### Problem 2(a) 
Split the data into a train and a test set. Use a random selection of 33% of the samples as test data. Sklearn provides the [`train_test_split`](http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.train_test_split.html) function for this purpose. Print the dimensions of all the train and test data sets you have created. 

In [408]:
### Your code here ###

#### Problem 2(b) 

 Use ***ten fold cross validation*** to estimate the optimal value for $k$ for the iris data set. 

**Note**: For your convenience sklearn does not only include the [KNN classifier](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html), but also a [grid search function](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html#sklearn.grid_search.GridSearchCV). The function is called grid search, because if you have to optimize more than one parameter, it is common practice to define a range of possible values for each parameter. An exhaustive search then runs over the complete grid defined by all the possible parameter combinations. This can get very computation heavy, but luckily our KNN classifier only requires tuning of a single parameter for this problem set. 

In [409]:
### Your code here ###

#### Problem 2(c)

Visualize the result by plotting the score results versus values for $k$. 

In [410]:
### Your code here ###

Verify that the grid search has indeed chosen the right parameter value for $k$.

In [411]:
### Your code here ###

#### Problem 2(d)

Test the performance of our tuned KNN classifier on the test set.

In [412]:
### Your code here ###

## Discussion for Problem 2

*Write a brief discussion of your conclusions to the questions and tasks above in 100 words or less.*

---

## Problem 3: Supervised Learning - Estimating Boston house pricing using Linear Regression and Regression Trees

##### Import the Boston House Pricing Dataset; it comes with the scikit or you can dowload it FROM UCI ML housing dataset. (http://archive.ics.uci.edu/ml/datasets/Housing)

In [413]:
### Your code here ###

#### Find the mostimportant features

In [414]:
### Your code here ###

#### Using 10-fold cross validation separate the test and training data sets

In [415]:
### Your code here ###

In [416]:
Normalize the data

SyntaxError: invalid syntax (<ipython-input-416-20b88cf8a636>, line 1)

In [ ]:
### Your code here ###

### Start with a lineal model and evaluate how well it can predict the price variable

In [ ]:
### Your code here ###

### Try using Ridge regression and evaluate the result of the 10-fold cross-validation

In [ ]:
### Your code here ###

### Train the Regression Tree and evaluate using 10-fold cross validation; Specify the parapmeters used and how you changed them to increase the accuracy;  

In [ ]:
### Your code here ###

### Discussion for Problem 3  Write a brief discussion of your conclusions to the questions and tasks above in 100 words or less.Compare all three aproaches and discuss your findings in 100 words or less